In [1]:
#champion analysis -- try to get the winrates

#this code creates and cleans the data and uploads it to a mySQL database;
#import dependencies
# Dependencies
import json
import requests
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import time
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

In [2]:
df_1 = pd.read_excel('2019_summer.xlsx', sheet_name='Sheet1')

In [3]:
df_1.columns

Index(['gameid', 'url', 'league', 'split', 'date', 'week', 'game', 'patchno',
       'playerid', 'side', 'position', 'player', 'team', 'champion', 'ban1',
       'ban2', 'ban3', 'ban4', 'ban5', 'gamelength', 'result', 'k', 'd', 'a',
       'teamkills', 'teamdeaths', 'doubles', 'triples', 'quadras', 'pentas',
       'fb', 'fbassist', 'fbvictim', 'fbtime', 'kpm', 'okpm', 'ckpm', 'fd',
       'fdtime', 'teamdragkills', 'oppdragkills', 'elementals',
       'oppelementals', 'firedrakes', 'waterdrakes', 'earthdrakes',
       'airdrakes', 'elders', 'oppelders', 'herald', 'heraldtime', 'ft',
       'fttime', 'firstmidouter', 'firsttothreetowers', 'teamtowerkills',
       'opptowerkills', 'fbaron', 'fbarontime', 'teambaronkills',
       'oppbaronkills', 'dmgtochamps', 'dmgtochampsperminute', 'dmgshare',
       'earnedgoldshare', 'wards', 'wpm', 'wardshare', 'wardkills', 'wcpm',
       'visionwards', 'visionwardbuys', 'visiblewardclearrate',
       'invisiblewardclearrate', 'totalgold', 'earnedg

In [4]:
champ_book = df_1[['gameid']]

stats_book = df_1[['gameid', 'side', 'result', 'position', 'champion',
                  'csat15', 'oppcsat15', 'csdat15', 'k', 'd', 'a',
                  'totalgold', 'dmgtochamps',]]

stats_book.head()

,gameid,side,result,position,champion,csat15,oppcsat15,csdat15,k,d,a,totalgold,dmgtochamps
0,1092338,Blue,1,Top,Aatrox,127.0,152.0,-25.0,6,3,12,15443.0,19265.0
1,1092338,Blue,1,Jungle,Hecarim,110.0,93.0,17.0,1,5,10,12741.0,8693.0
2,1092338,Blue,1,Middle,Sylas,139.0,145.0,-6.0,9,2,7,16317.0,18059.0
3,1092338,Blue,1,ADC,Varus,148.0,122.0,26.0,5,2,9,14937.0,17891.0
4,1092338,Blue,1,Support,Galio,24.0,23.0,1.0,0,2,14,9584.0,6637.0


In [5]:
#remove duplicates

champ_book.drop_duplicates(subset='gameid', inplace=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
#create a list for unique game ids that I can loop through 
game_ids = []

for row in champ_book['gameid']:
    game_ids.append(row)


In [7]:
df_1.columns

Index(['gameid', 'url', 'league', 'split', 'date', 'week', 'game', 'patchno',
       'playerid', 'side', 'position', 'player', 'team', 'champion', 'ban1',
       'ban2', 'ban3', 'ban4', 'ban5', 'gamelength', 'result', 'k', 'd', 'a',
       'teamkills', 'teamdeaths', 'doubles', 'triples', 'quadras', 'pentas',
       'fb', 'fbassist', 'fbvictim', 'fbtime', 'kpm', 'okpm', 'ckpm', 'fd',
       'fdtime', 'teamdragkills', 'oppdragkills', 'elementals',
       'oppelementals', 'firedrakes', 'waterdrakes', 'earthdrakes',
       'airdrakes', 'elders', 'oppelders', 'herald', 'heraldtime', 'ft',
       'fttime', 'firstmidouter', 'firsttothreetowers', 'teamtowerkills',
       'opptowerkills', 'fbaron', 'fbarontime', 'teambaronkills',
       'oppbaronkills', 'dmgtochamps', 'dmgtochampsperminute', 'dmgshare',
       'earnedgoldshare', 'wards', 'wpm', 'wardshare', 'wardkills', 'wcpm',
       'visionwards', 'visionwardbuys', 'visiblewardclearrate',
       'invisiblewardclearrate', 'totalgold', 'earnedg

In [8]:
#write code to get champ stats on a per-matchup basis;

game_number = []
game_result = []
position = []
match_side = []
champs_winner = []
champs_loser = []
champion_win = []
champion_loss = []
kills = []
deaths = []
assists = []
csat15 = []
oppcsat15 = []
total_gold = []
damage = []


for game in game_ids:
    match = stats_book.loc[stats_book['gameid'] == game]
    top = match.loc[match['position'] == 'Top']
    jungle = match.loc[match['position'] == 'Jungle']
    mid = match.loc[match['position'] == 'Middle']
    bot = match.loc[match['position'] == 'ADC']
    support = match.loc[match['position'] == 'Support']
#     game_id.append(row['gameid'])
#     top_champ = top['champion']
    for index, row in top.iterrows():
        if row['result'] == 1:
            game_number.append(row['gameid'])
            game_result.append(row['result'])
            position.append(row['position'])
            match_side.append(row['side'])
            champs_winner.append(row['champion'])
            kills.append(row['k'])
            deaths.append(row['d'])
            assists.append(row['a'])
            csat15.append(row['csat15'])
            oppcsat15.append(row['oppcsat15'])
            total_gold.append(row['totalgold'])
            damage.append(row['dmgtochamps'])
        if row['result'] == 0:
            champs_loser.append(row['champion'])
        if row['result'] == 1:
            champion_win.append(row['result'])
        if row['result'] == 0:
            champion_loss.append(row['result'])
    for index, row in jungle.iterrows():
        if row['result'] == 1:
            game_number.append(row['gameid'])
            game_result.append(row['result'])
            position.append(row['position'])
            match_side.append(row['side'])
            champs_winner.append(row['champion'])
            kills.append(row['k'])
            deaths.append(row['d'])
            assists.append(row['a'])
            csat15.append(row['csat15'])
            oppcsat15.append(row['oppcsat15'])
            total_gold.append(row['totalgold'])
            damage.append(row['dmgtochamps'])
        if row['result'] == 0:
            champs_loser.append(row['champion'])
        if row['result'] == 1:
            champion_win.append(row['result'])
        if row['result'] == 0:
            champion_loss.append(row['result'])
    for index, row in mid.iterrows():
        if row['result'] == 1:
            game_number.append(row['gameid'])
            game_result.append(row['result'])
            position.append(row['position'])
            match_side.append(row['side'])
            champs_winner.append(row['champion'])
            kills.append(row['k'])
            deaths.append(row['d'])
            assists.append(row['a'])
            csat15.append(row['csat15'])
            oppcsat15.append(row['oppcsat15'])
            total_gold.append(row['totalgold'])
            damage.append(row['dmgtochamps'])
        if row['result'] == 0:
            champs_loser.append(row['champion'])
        if row['result'] == 1:
            champion_win.append(row['result'])
        if row['result'] == 0:
            champion_loss.append(row['result'])
    for index, row in bot.iterrows():
        if row['result'] == 1:
            game_number.append(row['gameid'])
            game_result.append(row['result'])
            position.append(row['position'])
            match_side.append(row['side'])
            champs_winner.append(row['champion'])
            kills.append(row['k'])
            deaths.append(row['d'])
            assists.append(row['a'])
            csat15.append(row['csat15'])
            oppcsat15.append(row['oppcsat15'])
            total_gold.append(row['totalgold'])
            damage.append(row['dmgtochamps'])
        if row['result'] == 0:
            champs_loser.append(row['champion'])
        if row['result'] == 1:
            champion_win.append(row['result'])
        if row['result'] == 0:
            champion_loss.append(row['result'])
    for index, row in support.iterrows():
        if row['result'] == 1:
            game_number.append(row['gameid'])
            game_result.append(row['result'])
            position.append(row['position'])
            match_side.append(row['side'])
            champs_winner.append(row['champion'])
            kills.append(row['k'])
            deaths.append(row['d'])
            assists.append(row['a'])
            csat15.append(row['csat15'])
            oppcsat15.append(row['oppcsat15'])
            total_gold.append(row['totalgold'])
            damage.append(row['dmgtochamps'])
        if row['result'] == 0:
            champs_loser.append(row['champion'])
        if row['result'] == 1:
            champion_win.append(row['result'])
        if row['result'] == 0:
            champion_loss.append(row['result'])
    




In [9]:
# create a dataframe to find the average gold and stats on a per-matchup basis

major_stats_dict = {
    "winner": champs_winner,
    "loser": champs_loser,
    'kills' : kills,
    'deaths' : deaths,
    'assists' : assists,
    'csat15' : csat15,
    'oppcsat15' : oppcsat15,
    'total_gold' : total_gold,
    'damage' : damage 
}

major_stats_df = pd.DataFrame(major_stats_dict)

major_stats_df.head()


,winner,loser,kills,deaths,assists,csat15,oppcsat15,total_gold,damage
0,Aatrox,Irelia,6,3,12,127.0,152.0,15443.0,19265.0
1,Hecarim,Vi,1,5,10,110.0,93.0,12741.0,8693.0
2,Sylas,Ryze,9,2,7,139.0,145.0,16317.0,18059.0
3,Varus,Xayah,5,2,9,148.0,122.0,14937.0,17891.0
4,Galio,Braum,0,2,14,24.0,23.0,9584.0,6637.0


In [10]:
# add matchup row so i can merge/join later

major_stats_df['matchup'] = major_stats_df.winner.astype(str).str.cat(major_stats_df.loser.astype(str), sep=" , ")

major_stats_df.dropna()

average_stats_df = major_stats_df.groupby(['matchup']).mean()

cleaned_average_stats_df = average_stats_df.reset_index()

cleaned_average_stats_df.head()


,matchup,kills,deaths,assists,csat15,oppcsat15,total_gold,damage
0,"Aatrox , Akali",3.333333,2.666667,8.333333,126.666667,136.000000,15355.000000,16049.333333
1,"Aatrox , Azir",4.333333,0.666667,9.000000,129.333333,137.666667,14094.666667,12629.000000
2,"Aatrox , Camille",4.285714,1.000000,4.714286,131.800000,124.000000,12278.800000,12691.200000
3,"Aatrox , Fiora",3.428571,1.142857,4.428571,129.250000,122.750000,13688.000000,14177.000000
4,"Aatrox , Gangplank",3.727273,1.272727,4.909091,139.375000,112.750000,15388.250000,17006.375000


In [11]:
#create dataframe to find the matchup stats on a per-matchup basis

champ_dict = {
    "winning_champion": champs_winner,
    "win": champion_win,
    "losing_champion": champs_loser,
    "loss": champion_loss
}

champion_df = pd.DataFrame(champ_dict)

champion_df.head()


,winning_champion,win,losing_champion,loss
0,Aatrox,1,Irelia,0
1,Hecarim,1,Vi,0
2,Sylas,1,Ryze,0
3,Varus,1,Xayah,0
4,Galio,1,Braum,0


In [49]:
champion_df.head()

,winning_champion,win,losing_champion,loss
0,Aatrox,1,Irelia,0
1,Hecarim,1,Vi,0
2,Sylas,1,Ryze,0
3,Varus,1,Xayah,0
4,Galio,1,Braum,0


In [50]:
#test to try to groupby two columns;

total_wins = champion_df.groupby(['winning_champion', 'losing_champion']).count()

total_wins.head()

win  loss
winning_champion losing_champion           
Aatrox           Akali              3     3
                 Azir               3     3
                 Camille            7     7
                 Fiora              7     7
                 Gangplank         11    11

In [14]:
total_losses = champion_df.groupby(['losing_champion', 'winning_champion']).count()

total_losses.head()

win  loss
losing_champion winning_champion           
Aatrox          Akali               4     4
                Azir                1     1
                Camille            10    10
                Corki               5     5
                Elise               1     1

In [15]:
total_wins['loss'] = total_losses['loss']

total_wins.head()

win  loss
winning_champion losing_champion           
Aatrox           Akali              3   4.0
                 Azir               3   1.0
                 Camille            7  10.0
                 Fiora              7   6.0
                 Gangplank         11   5.0

In [16]:
total_wins['total_matches'] = total_wins['win'] + total_wins['loss']

total_wins['matchup_strength'] = (total_wins['win'] - total_wins['loss'])*2

total_wins.head()

win  loss  total_matches  matchup_strength
winning_champion losing_champion                                            
Aatrox           Akali              3   4.0            7.0              -2.0
                 Azir               3   1.0            4.0               4.0
                 Camille            7  10.0           17.0              -6.0
                 Fiora              7   6.0           13.0               2.0
                 Gangplank         11   5.0           16.0              12.0

In [17]:
#drop NaN values

champion_stats_book = total_wins.dropna()

# champion_stats_book.drop(index=0, columns='win')

# # master_df = champion_stats_book.transpose()

# # test = master_df.reset_index()

# # master_df.head()

champion_stats_book.head()

win  loss  total_matches  matchup_strength
winning_champion losing_champion                                            
Aatrox           Akali              3   4.0            7.0              -2.0
                 Azir               3   1.0            4.0               4.0
                 Camille            7  10.0           17.0              -6.0
                 Fiora              7   6.0           13.0               2.0
                 Gangplank         11   5.0           16.0              12.0

In [18]:
############ testing machine learning

matchups = []
win_count = []
loss_count = []
matchup_strength = []

for key, value in champion_stats_book.iterrows():
    matchups.append(key)
    win_count.append(value['win'])
    loss_count.append(value['loss'])
    matchup_strength.append(value['matchup_strength'])
    

matchups_df = pd.DataFrame(matchups)

stats_book = {
    'win': win_count,
    'loss': loss_count,
    'matchup_strength': matchup_strength   
}

stats_df = pd.DataFrame(stats_book)

stats_df.head()

,win,loss,matchup_strength
0,3.0,4.0,-2.0
1,3.0,1.0,4.0
2,7.0,10.0,-6.0
3,7.0,6.0,2.0
4,11.0,5.0,12.0


In [19]:
master_stats_df = pd.concat([matchups_df, stats_df], axis=1)

master_stats_df.rename(columns={0:'winner'}, inplace=True)

master_stats_df.rename(columns={1:'loser'}, inplace=True)

master_stats_df.head()

## save this to a csv to pull into my model

# Export file as a CSV, without the Pandas index, but with the header
master_stats_df.to_csv("champion_stats_a.csv", index=False, header=True)

# cleaned_average_stats_df.head()

cleaned_average_stats_df.to_csv("champion_stats_b.csv", index=False, header=True)

## save this to a csv as well

In [20]:
# create a new column w/ a combined matchup

master_stats_df['matchup'] = master_stats_df.winner.astype(str).str.cat(master_stats_df.loser.astype(str), sep=" , ")

cleaned_master_stats_df = master_stats_df.drop(columns=['winner', 'loser'])

cleaned_master_stats_df.head()



,win,loss,matchup_strength,matchup
0,3.0,4.0,-2.0,"Aatrox , Akali"
1,3.0,1.0,4.0,"Aatrox , Azir"
2,7.0,10.0,-6.0,"Aatrox , Camille"
3,7.0,6.0,2.0,"Aatrox , Fiora"
4,11.0,5.0,12.0,"Aatrox , Gangplank"


In [21]:
main_stats_dict = {
    "game_id": game_number, 
    "winner": champs_winner,
    "loser": champs_loser,
    "side": match_side,
    "result": game_result,
    "position": position
}

main_stats_df = pd.DataFrame(main_stats_dict)

main_stats_df.head()


,game_id,winner,loser,side,result,position
0,1092338,Aatrox,Irelia,Blue,1,Top
1,1092338,Hecarim,Vi,Blue,1,Jungle
2,1092338,Sylas,Ryze,Blue,1,Middle
3,1092338,Varus,Xayah,Blue,1,ADC
4,1092338,Galio,Braum,Blue,1,Support


In [22]:
main_stats_df['matchup'] = main_stats_df.winner.astype(str).str.cat(main_stats_df.loser.astype(str), sep=" , ")

cleaned_main_stats_df = main_stats_df.drop(columns=['winner', 'loser'])

cleaned_average_stats_df.head()


,matchup,kills,deaths,assists,csat15,oppcsat15,total_gold,damage
0,"Aatrox , Akali",3.333333,2.666667,8.333333,126.666667,136.000000,15355.000000,16049.333333
1,"Aatrox , Azir",4.333333,0.666667,9.000000,129.333333,137.666667,14094.666667,12629.000000
2,"Aatrox , Camille",4.285714,1.000000,4.714286,131.800000,124.000000,12278.800000,12691.200000
3,"Aatrox , Fiora",3.428571,1.142857,4.428571,129.250000,122.750000,13688.000000,14177.000000
4,"Aatrox , Gangplank",3.727273,1.272727,4.909091,139.375000,112.750000,15388.250000,17006.375000


In [23]:
master_merged_stats_book = pd.merge(cleaned_main_stats_df, cleaned_master_stats_df, on='matchup')
    
master_merged_stats_book.head()

,game_id,side,result,position,matchup,win,loss,matchup_strength
0,1092338,Blue,1,Top,"Aatrox , Irelia",10.0,5.0,10.0
1,1132838,Red,1,Top,"Aatrox , Irelia",10.0,5.0,10.0
2,1144254,Red,1,Top,"Aatrox , Irelia",10.0,5.0,10.0
3,1090924,Red,1,Top,"Aatrox , Irelia",10.0,5.0,10.0
4,1091677,Red,1,Middle,"Aatrox , Irelia",10.0,5.0,10.0


In [24]:
#goto create one column with merged side and result and define as blue win OR blue loss (dependent variable)

complete_merged_df = pd.merge(cleaned_average_stats_df, master_merged_stats_book, on='matchup')

# pre_cleaned_df.head()

# complete_merged_df = pre_cleaned_df.dropna()

In [25]:
#now merge the side/result columns

complete_merged_df['blue_result'] = complete_merged_df.side.astype(str).str.cat(complete_merged_df.result.astype(str), sep=" , ")

complete_merged_df.replace('Blue , 0', 'blue loss', inplace=True)
complete_merged_df.replace('Blue , 1', 'blue win', inplace=True)
complete_merged_df.replace('Red , 1', 'blue loss', inplace=True)
complete_merged_df.replace('Red , 0', 'blue win', inplace=True)

complete_merged_df.head()


,matchup,kills,deaths,assists,csat15,oppcsat15,total_gold,damage,game_id,side,result,position,win,loss,matchup_strength,blue_result
0,"Aatrox , Akali",3.333333,2.666667,8.333333,126.666667,136.000000,15355.000000,16049.333333,1131400,Blue,1,Top,3.0,4.0,-2.0,blue win
1,"Aatrox , Akali",3.333333,2.666667,8.333333,126.666667,136.000000,15355.000000,16049.333333,1144855,Blue,1,Top,3.0,4.0,-2.0,blue win
2,"Aatrox , Akali",3.333333,2.666667,8.333333,126.666667,136.000000,15355.000000,16049.333333,1092075,Blue,1,Top,3.0,4.0,-2.0,blue win
3,"Aatrox , Azir",4.333333,0.666667,9.000000,129.333333,137.666667,14094.666667,12629.000000,1092418,Red,1,Middle,3.0,1.0,4.0,blue loss
4,"Aatrox , Azir",4.333333,0.666667,9.000000,129.333333,137.666667,14094.666667,12629.000000,1124632,Blue,1,Middle,3.0,1.0,4.0,blue win


In [26]:
#locate all rows w/ null values

nulls = complete_merged_df[complete_merged_df.isna().any(axis=1)]

nulls_games = []

index_names = []

for key, row in nulls.iterrows():
    nulls_games.append(row['game_id'])
    
for game in nulls_games:
    for key, row in complete_merged_df.iterrows():
        if row['game_id'] == game:
            index_names.append(key)
        
            


In [27]:
test_final = complete_merged_df.drop(index_names)

In [28]:
print(len(complete_merged_df['game_id']))

2354


In [29]:
print(len(test_final['game_id']))

2008


In [30]:
#now try to transpose all of the matchups so they all are on one row by game id

master_game_ids = []


for row in test_final['game_id']:
    if row not in master_game_ids:
        master_game_ids.append(row)



print(len(master_game_ids))

485


In [31]:
#now write a new dataframe that will transpose the data
#goto append the result ONCE for each game and the game id

#store top results;
top_id = []
top_result = []
top_matchup = []
top_k = []
top_a = []
top_d = []
top_cs = []
top_op_cs = []
top_gold = []
top_damage = []
top_matchup_strength = []

#store jungle results; 
jg_id = []
jg_result = []
jg_matchup = []
jg_k = []
jg_a = []
jg_d = []
jg_cs = []
jg_op_cs = []
jg_gold = []
jg_damage = []
jg_matchup_strength = []

#store mid results; 
mid_id = []
mid_result = []
mid_matchup = []
mid_k = []
mid_a = []
mid_d = []
mid_cs = []
mid_op_cs = []
mid_gold = []
mid_damage = []
mid_matchup_strength = []

#store bot results; 
bot_id = []
bot_result = []
bot_matchup = []
bot_k = []
bot_a = []
bot_d = []
bot_cs = []
bot_op_cs = []
bot_gold = []
bot_damage = []
bot_matchup_strength = []

#store sup results; 
sup_id = []
sup_result = []
sup_matchup = []
sup_k = []
sup_a = []
sup_d = []
sup_cs = []
sup_op_cs = []
sup_gold = []
sup_damage = []
sup_matchup_strength = []


for game in master_game_ids:
    match = test_final.loc[test_final['game_id'] == game]
    top = match.loc[match['position'] == 'Top']
    jungle = match.loc[match['position'] == 'Jungle']
    mid = match.loc[match['position'] == 'Middle']
    bot = match.loc[match['position'] == 'ADC']
    support = match.loc[match['position'] == 'Support']
    for key, row in top.iterrows():
        top_id.append(row['game_id'])
        top_result.append(row['blue_result'])
        top_matchup.append(row['matchup'])
        top_k.append(row['kills'])
        top_a.append(row['assists'])
        top_d.append(row['deaths'])
        top_cs.append(row['csat15'])
        top_op_cs.append(row['oppcsat15'])
        top_gold.append(row['total_gold'])
        top_damage.append(row['damage'])
        top_matchup_strength.append(row['matchup_strength'])
    for key, row in jungle.iterrows():
        jg_id.append(row['game_id'])
        jg_result.append(row['blue_result'])
        jg_matchup.append(row['matchup'])
        jg_k.append(row['kills'])
        jg_a.append(row['assists'])
        jg_d.append(row['deaths'])
        jg_cs.append(row['csat15'])
        jg_op_cs.append(row['oppcsat15'])
        jg_gold.append(row['total_gold'])
        jg_damage.append(row['damage'])
        jg_matchup_strength.append(row['matchup_strength'])
    for key, row in mid.iterrows():
        mid_id.append(row['game_id'])
        mid_result.append(row['blue_result'])
        mid_matchup.append(row['matchup'])
        mid_k.append(row['kills'])
        mid_a.append(row['assists'])
        mid_d.append(row['deaths'])
        mid_cs.append(row['csat15'])
        mid_op_cs.append(row['oppcsat15'])
        mid_gold.append(row['total_gold'])
        mid_damage.append(row['damage'])
        mid_matchup_strength.append(row['matchup_strength'])
    for key, row in bot.iterrows():
        bot_id.append(row['game_id'])
        bot_result.append(row['blue_result'])
        bot_matchup.append(row['matchup'])
        bot_k.append(row['kills'])
        bot_a.append(row['assists'])
        bot_d.append(row['deaths'])
        bot_cs.append(row['csat15'])
        bot_op_cs.append(row['oppcsat15'])
        bot_gold.append(row['total_gold'])
        bot_damage.append(row['damage'])
        bot_matchup_strength.append(row['matchup_strength'])
    for key, row in support.iterrows():
        sup_id.append(row['game_id'])
        sup_result.append(row['blue_result'])
        sup_matchup.append(row['matchup'])
        sup_k.append(row['kills'])
        sup_a.append(row['assists'])
        sup_d.append(row['deaths'])
        sup_cs.append(row['csat15'])
        sup_op_cs.append(row['oppcsat15'])
        sup_gold.append(row['total_gold'])
        sup_damage.append(row['damage'])
        sup_matchup_strength.append(row['matchup_strength'])
    

In [32]:
#create the dataframe w/ long rows to use in the model

top_stats_dict = {
    'game_id': top_id,
    'top_result': top_result,
    'top_matchup': top_matchup,
    'top_k': top_k,
    'top_a': top_a,
    'top_d': top_d,
    'top_cs': top_cs,
    'top_op_cs': top_op_cs,
    'top_gold': top_gold,
    'top_damage': top_damage,
    'top_matchup_strength': top_matchup_strength,
    
}

top_stats_df = pd.DataFrame(top_stats_dict)

top_stats_df.head()

,game_id,top_result,top_matchup,top_k,top_a,top_d,top_cs,top_op_cs,top_gold,top_damage,top_matchup_strength
0,1131400,blue win,"Aatrox , Akali",3.333333,8.333333,2.666667,126.666667,136.0,15355.0,16049.333333,-2.0
1,1144855,blue win,"Aatrox , Akali",3.333333,8.333333,2.666667,126.666667,136.0,15355.0,16049.333333,-2.0
2,1092075,blue win,"Aatrox , Akali",3.333333,8.333333,2.666667,126.666667,136.0,15355.0,16049.333333,-2.0
3,1092418,blue loss,"Vladimir , Camille",1.000000,6.000000,2.000000,118.000000,116.0,10972.0,13027.000000,-4.0
4,1124632,blue win,"Renekton , Camille",3.000000,8.333333,1.333333,119.500000,117.5,13831.5,14174.000000,2.0


In [33]:
#jungle df

jg_stats_dict = {
    'game_id': jg_id,
    'jg_result': jg_result,
    'jg_matchup': jg_matchup,
    'jg_k': jg_k,
    'jg_a': jg_a,
    'jg_d': jg_d,
    'jg_cs': jg_cs,
    'jg_op_cs': jg_op_cs,
    'jg_gold': jg_gold,
    'jg_damage': jg_damage,
    'jg_matchup_strength': jg_matchup_strength
}

jg_stats_df = pd.DataFrame(jg_stats_dict)

jg_stats_df.head()

,game_id,jg_result,jg_matchup,jg_k,jg_a,jg_d,jg_cs,jg_op_cs,jg_gold,jg_damage,jg_matchup_strength
0,1131400,blue win,"Skarner , Rek'Sai",1.916667,7.166667,0.916667,90.000000,93.583333,11273.416667,4207.666667,22.0
1,1144855,blue win,"Jarvan IV , Hecarim",4.000000,9.500000,3.500000,101.000000,107.500000,13534.000000,8771.000000,2.0
2,1092075,blue win,"Gragas , Jarvan IV",2.250000,7.500000,1.500000,83.000000,91.666667,11376.000000,10066.333333,-6.0
3,1092418,blue loss,"Karthus , Sejuani",2.666667,10.666667,3.666667,110.333333,79.666667,12668.666667,18958.000000,-6.0
4,1124632,blue win,"Gragas , Sylas",5.000000,9.500000,2.000000,98.000000,87.000000,15218.000000,24118.000000,0.0


In [34]:
#create mid stats df

mid_stats_dict = {
    'game_id': mid_id,
    'mid_result': mid_result,
    'mid_matchup': mid_matchup,
    'mid_k': mid_k,
    'mid_a': mid_a,
    'mid_d': mid_d,
    'mid_cs': mid_cs,
    'mid_op_cs': mid_op_cs,
    'mid_gold': mid_gold,
    'mid_damage': mid_damage,
    'mid_matchup_strength': mid_matchup_strength
}

mid_stats_df = pd.DataFrame(mid_stats_dict)

mid_stats_df.head()



,game_id,mid_result,mid_matchup,mid_k,mid_a,mid_d,mid_cs,mid_op_cs,mid_gold,mid_damage,mid_matchup_strength
0,1131400,blue win,"Lissandra , Corki",1.000000,8.0,2.000000,154.000000,149.000000,16409.000000,16032.0,-2.0
1,1144855,blue win,"Ryze , Sylas",4.500000,5.0,1.833333,138.800000,130.400000,14854.200000,14083.0,-10.0
2,1092075,blue win,"Viktor , Sylas",3.500000,6.0,2.000000,112.500000,125.000000,15257.500000,21118.5,2.0
3,1092418,blue loss,"Aatrox , Azir",4.333333,9.0,0.666667,129.333333,137.666667,14094.666667,12629.0,4.0
4,1124632,blue win,"Aatrox , Azir",4.333333,9.0,0.666667,129.333333,137.666667,14094.666667,12629.0,4.0


In [35]:
#create bot stats df

bot_stats_dict = {
    'game_id': bot_id,
    'bot_result': bot_result,
    'bot_matchup': bot_matchup,
    'bot_k': bot_k,
    'bot_a': bot_a,
    'bot_d': bot_d,
    'bot_cs': bot_cs,
    'bot_op_cs': bot_op_cs,
    'bot_gold': bot_gold,
    'bot_damage': bot_damage,
    'bot_matchup_strength': bot_matchup_strength
}

bot_stats_df = pd.DataFrame(bot_stats_dict)

bot_stats_df.head()



,game_id,bot_result,bot_matchup,bot_k,bot_a,bot_d,bot_cs,bot_op_cs,bot_gold,bot_damage,bot_matchup_strength
0,1131400,blue win,"Xayah , Lucian",4.800000,5.600000,1.200000,129.428571,133.00,16657.428571,16006.714286,6.0
1,1144855,blue win,"Kai'Sa , Xayah",6.777778,6.277778,1.555556,132.625000,135.25,17546.750000,21408.750000,10.0
2,1092075,blue win,"Xayah , Ashe",6.000000,8.000000,1.500000,144.000000,134.00,17281.000000,30339.000000,-4.0
3,1124632,blue win,"Varus , Lucian",3.500000,8.500000,2.000000,137.000000,143.00,14106.000000,14910.000000,0.0
4,1118773,blue win,"Lucian , Kai'Sa",4.500000,6.500000,1.666667,144.250000,116.00,15584.500000,13585.000000,-10.0


In [36]:
#create sup stats df

sup_stats_dict = {
    'game_id': sup_id,
    'sup_result': sup_result,
    'sup_matchup': sup_matchup,
    'sup_k': sup_k,
    'sup_a': sup_a,
    'sup_d': sup_d,
    'sup_cs': sup_cs,
    'sup_op_cs': sup_op_cs,
    'sup_gold': sup_gold,
    'sup_damage': sup_damage,
    'sup_matchup_strength': sup_matchup_strength
}

sup_stats_df = pd.DataFrame(sup_stats_dict)

sup_stats_df

,game_id,sup_result,sup_matchup,sup_k,sup_a,sup_d,sup_cs,sup_op_cs,sup_gold,sup_damage,sup_matchup_strength
0,1131400,blue win,"Lux , Nautilus",1.350000,11.050000,1.750000,7.785714,23.428571,9414.214286,7493.714286,22.0
1,1144855,blue win,"Galio , Rakan",0.500000,9.666667,3.000000,25.750000,6.000000,8316.000000,4139.750000,-2.0
2,1092075,blue win,"Rakan , Tahm Kench",0.800000,11.600000,1.466667,5.692308,28.000000,9215.000000,6079.769231,2.0
3,1092418,blue loss,"Yuumi , Alistar",1.500000,13.250000,1.000000,2.500000,21.500000,8346.000000,10330.000000,-6.0
4,1124632,blue win,"Braum , Nautilus",0.500000,11.700000,2.600000,24.500000,25.375000,9528.750000,4912.625000,2.0
5,1121998,blue loss,"Nautilus , Tahm Kench",0.750000,9.375000,2.875000,26.166667,32.333333,8197.666667,4206.500000,0.0
6,1123988,blue win,"Lux , Tahm Kench",1.000000,11.125000,1.083333,6.666667,28.933333,8279.200000,7132.000000,2.0
7,1114480,blue win,"Yuumi , Lux",1.294118,11.705882,0.588235,3.083333,7.166667,9607.416667,14417.833333,-14.0
8,1124472,blue loss,"Lux , Nautilus",1.350000,11.050000,1.750000,7.785714,23.428571,9414.214286,7493.714286,22.0
9,2019-272Vici GamingInvictus Gaming,blue win,"Tahm Kench , Braum",0.800000,8.200000,1.750000,39.166667,24.333333,9057.944444,4833.388889,2.0


In [37]:
merge_jg = pd.merge(jg_stats_df, top_stats_df, on='game_id')
    
merge_jg.head()

,game_id,jg_result,jg_matchup,jg_k,jg_a,jg_d,jg_cs,jg_op_cs,jg_gold,jg_damage,...,top_result,top_matchup,top_k,top_a,top_d,top_cs,top_op_cs,top_gold,top_damage,top_matchup_strength
0,1131400,blue win,"Skarner , Rek'Sai",1.916667,7.166667,0.916667,90.000000,93.583333,11273.416667,4207.666667,...,blue win,"Aatrox , Akali",3.333333,8.333333,2.666667,126.666667,136.0,15355.0,16049.333333,-2.0
1,1144855,blue win,"Jarvan IV , Hecarim",4.000000,9.500000,3.500000,101.000000,107.500000,13534.000000,8771.000000,...,blue win,"Aatrox , Akali",3.333333,8.333333,2.666667,126.666667,136.0,15355.0,16049.333333,-2.0
2,1092075,blue win,"Gragas , Jarvan IV",2.250000,7.500000,1.500000,83.000000,91.666667,11376.000000,10066.333333,...,blue win,"Aatrox , Akali",3.333333,8.333333,2.666667,126.666667,136.0,15355.0,16049.333333,-2.0
3,1092418,blue loss,"Karthus , Sejuani",2.666667,10.666667,3.666667,110.333333,79.666667,12668.666667,18958.000000,...,blue loss,"Vladimir , Camille",1.000000,6.000000,2.000000,118.000000,116.0,10972.0,13027.000000,-4.0
4,1124632,blue win,"Gragas , Sylas",5.000000,9.500000,2.000000,98.000000,87.000000,15218.000000,24118.000000,...,blue win,"Renekton , Camille",3.000000,8.333333,1.333333,119.500000,117.5,13831.5,14174.000000,2.0


In [38]:
merge_mid = pd.merge(merge_jg, mid_stats_df, on='game_id')

merge_mid.head()

,game_id,jg_result,jg_matchup,jg_k,jg_a,jg_d,jg_cs,jg_op_cs,jg_gold,jg_damage,...,mid_result,mid_matchup,mid_k,mid_a,mid_d,mid_cs,mid_op_cs,mid_gold,mid_damage,mid_matchup_strength
0,1131400,blue win,"Skarner , Rek'Sai",1.916667,7.166667,0.916667,90.000000,93.583333,11273.416667,4207.666667,...,blue win,"Lissandra , Corki",1.000000,8.0,2.000000,154.000000,149.000000,16409.000000,16032.0,-2.0
1,1144855,blue win,"Jarvan IV , Hecarim",4.000000,9.500000,3.500000,101.000000,107.500000,13534.000000,8771.000000,...,blue win,"Ryze , Sylas",4.500000,5.0,1.833333,138.800000,130.400000,14854.200000,14083.0,-10.0
2,1092075,blue win,"Gragas , Jarvan IV",2.250000,7.500000,1.500000,83.000000,91.666667,11376.000000,10066.333333,...,blue win,"Viktor , Sylas",3.500000,6.0,2.000000,112.500000,125.000000,15257.500000,21118.5,2.0
3,1092418,blue loss,"Karthus , Sejuani",2.666667,10.666667,3.666667,110.333333,79.666667,12668.666667,18958.000000,...,blue loss,"Aatrox , Azir",4.333333,9.0,0.666667,129.333333,137.666667,14094.666667,12629.0,4.0
4,1124632,blue win,"Gragas , Sylas",5.000000,9.500000,2.000000,98.000000,87.000000,15218.000000,24118.000000,...,blue win,"Aatrox , Azir",4.333333,9.0,0.666667,129.333333,137.666667,14094.666667,12629.0,4.0


In [39]:
merge_bot = pd.merge(merge_mid, bot_stats_df, on='game_id')

merge_bot.head()

,game_id,jg_result,jg_matchup,jg_k,jg_a,jg_d,jg_cs,jg_op_cs,jg_gold,jg_damage,...,bot_result,bot_matchup,bot_k,bot_a,bot_d,bot_cs,bot_op_cs,bot_gold,bot_damage,bot_matchup_strength
0,1131400,blue win,"Skarner , Rek'Sai",1.916667,7.166667,0.916667,90.0,93.583333,11273.416667,4207.666667,...,blue win,"Xayah , Lucian",4.800000,5.600000,1.200000,129.428571,133.00,16657.428571,16006.714286,6.0
1,1144855,blue win,"Jarvan IV , Hecarim",4.000000,9.500000,3.500000,101.0,107.500000,13534.000000,8771.000000,...,blue win,"Kai'Sa , Xayah",6.777778,6.277778,1.555556,132.625000,135.25,17546.750000,21408.750000,10.0
2,1092075,blue win,"Gragas , Jarvan IV",2.250000,7.500000,1.500000,83.0,91.666667,11376.000000,10066.333333,...,blue win,"Xayah , Ashe",6.000000,8.000000,1.500000,144.000000,134.00,17281.000000,30339.000000,-4.0
3,1124632,blue win,"Gragas , Sylas",5.000000,9.500000,2.000000,98.0,87.000000,15218.000000,24118.000000,...,blue win,"Varus , Lucian",3.500000,8.500000,2.000000,137.000000,143.00,14106.000000,14910.000000,0.0
4,1118773,blue win,"Xin Zhao , Sylas",2.333333,8.000000,1.666667,77.0,87.000000,11316.000000,7001.000000,...,blue win,"Lucian , Kai'Sa",4.500000,6.500000,1.666667,144.250000,116.00,15584.500000,13585.000000,-10.0


In [40]:
the_real_df = pd.merge(merge_bot, sup_stats_df, on='game_id')
                     
the_real_df.head()

,game_id,jg_result,jg_matchup,jg_k,jg_a,jg_d,jg_cs,jg_op_cs,jg_gold,jg_damage,...,sup_result,sup_matchup,sup_k,sup_a,sup_d,sup_cs,sup_op_cs,sup_gold,sup_damage,sup_matchup_strength
0,1131400,blue win,"Skarner , Rek'Sai",1.916667,7.166667,0.916667,90.0,93.583333,11273.416667,4207.666667,...,blue win,"Lux , Nautilus",1.35,11.050000,1.750000,7.785714,23.428571,9414.214286,7493.714286,22.0
1,1144855,blue win,"Jarvan IV , Hecarim",4.000000,9.500000,3.500000,101.0,107.500000,13534.000000,8771.000000,...,blue win,"Galio , Rakan",0.50,9.666667,3.000000,25.750000,6.000000,8316.000000,4139.750000,-2.0
2,1092075,blue win,"Gragas , Jarvan IV",2.250000,7.500000,1.500000,83.0,91.666667,11376.000000,10066.333333,...,blue win,"Rakan , Tahm Kench",0.80,11.600000,1.466667,5.692308,28.000000,9215.000000,6079.769231,2.0
3,1124632,blue win,"Gragas , Sylas",5.000000,9.500000,2.000000,98.0,87.000000,15218.000000,24118.000000,...,blue win,"Braum , Nautilus",0.50,11.700000,2.600000,24.500000,25.375000,9528.750000,4912.625000,2.0
4,1121998,blue loss,"Skarner , Rek'Sai",1.916667,7.166667,0.916667,90.0,93.583333,11273.416667,4207.666667,...,blue loss,"Nautilus , Tahm Kench",0.75,9.375000,2.875000,26.166667,32.333333,8197.666667,4206.500000,0.0


In [41]:
# Export file as a CSV, without the Pandas index, but with the header
the_real_df.to_csv("champion_matchups_2019_table.csv", index=False, header=True)